In [1]:
import time
import os
import sys
import pandas as pd
from numpy import dtype
from glob import glob
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
#importação de dados
arquivos = sorted(glob(r'C:\Users\otavi\Desktop\base\*.csv'))
df_original = pd.concat((pd.read_csv(cont, 
    dtype={'CNES_EXECUTANTE':'str','IBGE_USUARIO':'str','CEP_USUARIO':'str', 'CEP_UNID_EXECUTANTE':'str'}) 
    for cont in arquivos),ignore_index=True)

In [3]:
df_cep = pd.read_csv(r'C:\Users\otavi\GitHub\SISREG_Estudo_caso-AI_FALTAS_USUARIOS\dados\TABELA_CEP_BR.txt', dtype=str, sep="	", header=None)
df_cep.columns = ['CEP','UF','CIDADE','BAIRRO','LOGRADOURO']  #http://cep.la/baixar

In [4]:
#criando um DataFrame somente os CEP's dos usuarios.
df_ceps_usuario = df_original['CEP_USUARIO']
df_ceps_usuario = pd.DataFrame(df_ceps_usuario)
df_ceps_usuario.drop_duplicates(inplace=True)
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

,CEP_USUARIO
0,12441170
1,12922810
2,66033540
3,54515090
4,88352495


In [5]:
#quantidade de CEP's localizado na base 
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

Cep's unicos de usuarios:  CEP_USUARIO    413882
dtype: int64


In [6]:
#localizando endereço do usuario com a base do CEP's BR
df_ceps_usuario = pd.merge(df_ceps_usuario, df_cep, left_on=['CEP_USUARIO'],right_on=['CEP'] , how='left')

In [7]:
#criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO']+ " - " + df_ceps_usuario['CIDADE']+ " - " + df_ceps_usuario['UF']
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

,CEP_USUARIO,CEP,UF,CIDADE,BAIRRO,LOGRADOURO,END_COMPLETO
0,12441170,12441170,SP,Pindamonhangaba,Jardim Padre Rodolfo,Rua Oswaldo Marcondes de Azevedo,Rua Oswaldo Marcondes de Azevedo - Pindamonhan...
1,12922810,12922810,SP,Bragança Paulista,Parque dos Estados,Rua Amapá,Rua Amapá - Bragança Paulista - SP
2,66033540,66033540,PA,Belém,Condor,Passagem Nazaré,Passagem Nazaré - Belém - PA
3,54515090,54515090,PE,Cabo de Santo Agostinho,COHAB,Rua Alexandra Figueiredo Silva,Rua Alexandra Figueiredo Silva - Cabo de Santo...
4,88352495,88352495,SC,Brusque,Limoeiro,Rua Itajaí,Rua Itajaí - Brusque - SC


In [8]:

driver = webdriver.Chrome(r"../navergadores/chromedriver.exe") #chromedriver.exe
url = "https://developers-dot-devsite-v2-prod.appspot.com/maps/documentation/utils/geocoder#q%3D"
driver.get(url)
time.sleep(5)

C:\Users\otavi\AppData\Local\Temp\ipykernel_19032\4258199709.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"../navergadores/chromedriver.exe") #chromedriver.exe


In [ ]:
for endereco in df_ceps_usuario:
    